In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df1 = pd.read_csv('eda/small1/labeled.csv')
df2 = pd.read_csv('eda/small2/labeled.csv')
df3 = pd.read_csv('eda/small3/labeled.csv')
df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
df.to_csv('data/merged_titles_labels.csv', index=False)
df.head()

,title,is_clickbait
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [10]:
df = preprocess_title(df)

In [11]:
from tqdm.notebook import tqdm
import time
# VECTOR_SIZE = 2500
# WINDOW = 4
EPOCHS = 500

# EPOCHS = 100
WORKERS = 10
MIN_COUNT = 1
# train many word2vec models with diferent VECTOR_SIZE and WINDOW
VECTOR_SIZEs = [100, 250, 500, 1000, 1500, 2000, 2500, 3000]
# VECTOR_SIZEs = [10]
# VECTOR_SIZEs = [i//10 for i in VECTOR_SIZEs]
WINDOWs = [3, 4, 5, 6, 7, 8]
# WINDOWs = [4]
SGs = [0, 1]

print('Start training')
# sleep 200 ms
time.sleep(0.2)

for VECTOR_SIZE in tqdm(VECTOR_SIZEs):
    print(f'Current VECTOR_SIZE: {VECTOR_SIZE}')
    for WINDOW in tqdm(WINDOWs, desc=f'WINDOW'):
        for sg in tqdm(SGs, desc=f'SG'):
            model = Word2Vec(df['title'], vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, sg=sg)
            model.train(df['title'], total_examples=len(df['title']), epochs=EPOCHS)
            model.save(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model')

Start training


  0%|          | 0/1 [00:00<?, ?it/s]

Current VECTOR_SIZE: 10


WINDOW:   0%|          | 0/1 [00:00<?, ?it/s]

SG:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
model = Word2Vec.load('word2vec_models/word2vec_vs100_win3_sg0.model')

In [12]:
df

,title,is_clickbait
0,"[house, dem, aide, even, see, comey, letter, j...",1
1,"[flynn, hillary, clinton, big, woman, campus, ...",0
2,"[truth, might, get, fired]",1
3,"[fifteen, civilians, killed, single, us, airst...",1
4,"[iranian, woman, jailed, fictional, unpublishe...",1
...,...,...
75017,"[make, female, hearts, flutter, iraq, throw, s...",0
75018,"[british, liberal, democrat, patsy, calton, fi...",0
75019,"[drone, smartphone, app, help, heart, attack, ...",0
75020,"[netanyahu, urges, pope, benedict, israel, den...",0


In [13]:
model.wv.most_similar('trump')

[('donald', 0.9674442410469055),
 ('obama', 0.9394640922546387),
 ('biden', 0.9286625385284424),
 ('trumps', 0.921497642993927),
 ('focused', 0.9203516244888306),
 ('electors', 0.9160240888595581),
 ('presidency', 0.9133554100990295),
 ('pbs', 0.9065513610839844),
 ('promised', 0.9027680158615112),
 ('commanderinchief', 0.8990459442138672)]